In [1]:
import pandas as pd
import numpy as np

In [2]:
def expectedScore(ratingA, ratingB):
    return 1 / (1 + 10 ** ((ratingB - ratingA) / 400))

In [39]:
def updateElo(ratingA, ratingB,scoreA,scoreB,baseKFactor=2):
    
    expectedA = expectedScore(ratingA, ratingB)
    expectedB = expectedScore(ratingB, ratingA)

    newRatingA = ratingA + baseKFactor * (scoreA - expectedA)
    newRatingB = ratingB + baseKFactor * (scoreB - expectedB)

    return round(newRatingA, 3), round(newRatingB, 3)

In [55]:
df = pd.read_csv('/Users/chinmaychoudhary/Documents/Repos/cricket/ball-by-ball/dataExtraction/ipl_match_by_match/335982.csv')
# df = df[df['year']==2008].reset_index(drop=True)
df

,inning,date,batter,nonStriker,bowler,over,ball,runs,batterRuns,extras,extrasType,wicket,playerOut,wicketKind,team,teams,venue
0,1,18-04-08,SC Ganguly,BB McCullum,P Kumar,1,1,1,0,1,{'legbyes': 1},0,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore V Kolkata Knight R...,M Chinnaswamy Stadium
1,1,18-04-08,BB McCullum,SC Ganguly,P Kumar,1,2,0,0,0,{},0,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore V Kolkata Knight R...,M Chinnaswamy Stadium
2,1,18-04-08,BB McCullum,SC Ganguly,P Kumar,1,3,1,0,1,{'wides': 1},0,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore V Kolkata Knight R...,M Chinnaswamy Stadium
3,1,18-04-08,BB McCullum,SC Ganguly,P Kumar,1,4,0,0,0,{},0,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore V Kolkata Knight R...,M Chinnaswamy Stadium
4,1,18-04-08,BB McCullum,SC Ganguly,P Kumar,1,5,0,0,0,{},0,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore V Kolkata Knight R...,M Chinnaswamy Stadium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2,18-04-08,P Kumar,SB Joshi,I Sharma,15,5,1,0,1,{'legbyes': 1},0,NaN,NaN,Royal Challengers Bangalore,Royal Challengers Bangalore V Kolkata Knight R...,M Chinnaswamy Stadium
221,2,18-04-08,SB Joshi,P Kumar,I Sharma,15,6,1,1,0,{},0,NaN,NaN,Royal Challengers Bangalore,Royal Challengers Bangalore V Kolkata Knight R...,M Chinnaswamy Stadium
222,2,18-04-08,P Kumar,SB Joshi,I Sharma,15,7,0,0,0,{},0,NaN,NaN,Royal Challengers Bangalore,Royal Challengers Bangalore V Kolkata Knight R...,M Chinnaswamy Stadium
223,2,18-04-08,SB Joshi,P Kumar,LR Shukla,16,1,1,0,1,{'wides': 1},0,NaN,NaN,Royal Challengers Bangalore,Royal Challengers Bangalore V Kolkata Knight R...,M Chinnaswamy Stadium


In [56]:
df.drop(['inning','nonStriker','over','ball','extrasType','playerOut','wicketKind','team','teams','venue'],axis=1,inplace=True)

In [57]:
df

,date,batter,bowler,runs,batterRuns,extras,wicket
0,18-04-08,SC Ganguly,P Kumar,1,0,1,0
1,18-04-08,BB McCullum,P Kumar,0,0,0,0
2,18-04-08,BB McCullum,P Kumar,1,0,1,0
3,18-04-08,BB McCullum,P Kumar,0,0,0,0
4,18-04-08,BB McCullum,P Kumar,0,0,0,0
...,...,...,...,...,...,...,...
220,18-04-08,P Kumar,I Sharma,1,0,1,0
221,18-04-08,SB Joshi,I Sharma,1,1,0,0
222,18-04-08,P Kumar,I Sharma,0,0,0,0
223,18-04-08,SB Joshi,LR Shukla,1,0,1,0


In [58]:
def generateScore(batterRuns, extras, wicket):
    RUN_WEIGHT = 1  # Weight for each run
    if batterRuns == 6:
        RUN_WEIGHT=2
    WICKET_WEIGHT = 15  # Equivalent runs for a wicket
    EXTRA_PENALTY = -1 # Penalty for each extra conceded

    batterScore = batterRuns * RUN_WEIGHT

    bowlerScore = (wicket * WICKET_WEIGHT) - (extras * EXTRA_PENALTY) - batterScore

    maxScore = max(batterScore, abs(bowlerScore))
    if maxScore > 0:
        normalizedBatterScore = batterScore / maxScore
        normalizedBowlerScore = bowlerScore / maxScore
    else:
        normalizedBatterScore = 0
        normalizedBowlerScore = 0

    return normalizedBatterScore, normalizedBowlerScore


In [59]:
df['batterElo'] = 1000
df['bowlerElo'] = 1000

In [70]:
batterEloDict = {batter:1000 for batter in list(df.batter.unique())}
bowlerEloDict = {bowler:1000 for bowler in list(df.bowler.unique())}

for index, row in df.iterrows():
    date = row['date']
    batter = row['batter']
    bowler = row['bowler']
    batterRuns = row['batterRuns']
    extras = row['extras']
    wicket = row['wicket']
    

    scoreA,scoreB = generateScore(batterRuns,extras,wicket)
    batterElo, bowlerElo = updateElo(batterEloDict[batter], bowlerEloDict[bowler],scoreA,scoreB)

    # eloRatingMatch.append((matchCount[team1], team1, newEloTeam1,season))
    # eloRatingMatch.append((matchCount[team2], team2, newEloTeam2,season))

    # Set new Elo ratings in the DataFrame
    df.at[index, 'batterElo'] = batterElo
    df.at[index, 'bowlerElo'] = bowlerElo

    # Update current Elo ratings
    batterEloDict[batter] = batterElo
    bowlerEloDict[bowler] = bowlerElo
    

In [71]:
df

,date,batter,bowler,runs,batterRuns,extras,wicket,batterElo,bowlerElo
0,18-04-08,SC Ganguly,P Kumar,1,0,1,0,999.000,1001.000
1,18-04-08,BB McCullum,P Kumar,0,0,0,0,999.003,999.997
2,18-04-08,BB McCullum,P Kumar,1,0,1,0,998.006,1000.994
3,18-04-08,BB McCullum,P Kumar,0,0,0,0,997.015,999.985
4,18-04-08,BB McCullum,P Kumar,0,0,0,0,996.024,998.976
...,...,...,...,...,...,...,...,...,...
220,18-04-08,P Kumar,I Sharma,1,0,1,0,993.365,985.229
221,18-04-08,SB Joshi,I Sharma,1,1,0,0,999.764,982.268
222,18-04-08,P Kumar,I Sharma,0,0,0,0,992.333,981.300
223,18-04-08,SB Joshi,LR Shukla,1,0,1,0,998.727,988.096


In [72]:
batterEloData = list(batterEloDict.items())
batterEloDf = pd.DataFrame(batterEloData, columns=['batter', 'eloRating'])
batterEloDf.sort_values('eloRating',ascending=False)

,batter,eloRating
1,BB McCullum,1025.249
3,DJ Hussey,1002.973
5,R Dravid,1000.974
2,RT Ponting,1000.933
4,Mohammad Hafeez,1000.637
9,CL White,999.685
8,JH Kallis,998.847
11,B Akhil,997.898
14,Z Khan,997.727
15,SB Joshi,997.696


In [75]:
bowlerEloData = list(bowlerEloDict.items())
bowlerEloDf = pd.DataFrame(bowlerEloData, columns=['batter', 'eloRating'])
bowlerEloDf.sort_values('eloRating',ascending=False)

,batter,eloRating
10,LR Shukla,989.127
5,CL White,983.318
7,I Sharma,981.300
6,AB Dinda,974.554
8,AB Agarkar,969.110
9,SC Ganguly,967.903
0,P Kumar,962.110
4,SB Joshi,955.305
3,JH Kallis,953.055
2,AA Noffke,948.906
